In [1]:
import pandas as pd

def normalize_price(price, min_price, max_price):
    # 0 en kötü, 1 en iyi (ucuz)
    return 1 - ((price - min_price) / (max_price - min_price))

def is_touristic_area(district):
    touristic_districts = ["Kadıköy", "Beşiktaş", "Beyoğlu", "Fatih", "Şişli", "Üsküdar"]
    return district.strip().lower() in [d.lower() for d in touristic_districts]

def calculate_score(row, min_price, max_price):
    price_score = normalize_price(row['price'], min_price, max_price)
    bonus = 0.0

    # Turistik bölge bonusu
    if is_touristic_area(row['district']):
        bonus += 0.25

    # İsteğe bağlı diğer kriterler burada:
    # if row['building_age'] in ['0 Yaşında', '1 Yaşında', '2 Yaşında']:
    #     bonus += 0.1

    return round(price_score + bonus, 3)

def assign_personality_scores(df):
    # Temizlik
    df = df.dropna(subset=['price', 'district'])  # Eksik verileri at
    df['price'] = df['price'].astype(str).str.replace('.', '', regex=False).str.replace(',', '', regex=False).astype(float)

    # Normalize için gerekli sınırlar
    min_price = df['price'].min()
    max_price = df['price'].max()

    # Puan hesapla
    df['personality_score'] = df.apply(lambda row: calculate_score(row, min_price, max_price), axis=1)

    return df


In [17]:
import pandas as pd

# Veriyi oku
df = pd.read_csv("cleaned_data100.csv", encoding="utf-8")

# Fiyatları sayıya çevir
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Turistik ilçeler listesi
touristic_districts = ["Kadıköy", "Beşiktaş", "Şişli", "Fatih", "Beyoğlu", "Üsküdar", "Sarıyer", "Bakırköy"]

# Base score hesapla
def calculate_base_score(row):
    price = row['price']
    district = str(row.get('districk', '')).strip().title()

    # Fiyat kurallarına göre base score
    if price >= 75000:
        base_score = 1.0
    elif price <= 15000:
        base_score = 0.1
    else:
        base_score = 0.5  # Orta seviye için istersen başka bir değer verebilirsin

    # Turistik ilçedeyse bonus ekle
    if district in touristic_districts:
        base_score += 0.1

    # Skoru 1.0'ı geçmesin
    return min(base_score, 1.0)

# Her satır için score hesapla
df['score'] = df.apply(calculate_base_score, axis=1).round(3)

# Sonuçları yazdır
print(df[['districk', 'price', 'score']].describe())
print(df[['districk', 'price', 'score']])

# Yeni dosyaya kaydet
df.to_csv("cleaned_data_scored100.csv", index=False, encoding="utf-8-sig")

print("✅ Yeni kurallara göre puanlama tamamlandı.")


               price      score
count      99.000000  99.000000
mean    48747.454545   0.612121
std     45268.500769   0.235291
min     10000.000000   0.100000
25%     20000.000000   0.500000
50%     34999.000000   0.600000
75%     63750.000000   0.600000
max    330000.000000   1.000000
           districk  price  score
0          Sarıyer   41000    0.6
1          Üsküdar   32000    0.6
2          Kadıköy   47000    0.6
3           Beykoz   18000    0.5
4          Sarıyer   50000    0.6
..              ...    ...    ...
94        Beşiktaş   40000    0.6
95          Pendik   24500    0.5
96       Kağıthane   25000    0.5
97   Gaziosmanpaşa   13000    0.1
98         Kadıköy   55000    0.6

[99 rows x 3 columns]
✅ Yeni kurallara göre puanlama tamamlandı.
